In [91]:
#Create a sample of 100 from known 2016 Russian Internet Research Agency tweets; excluded retweets

import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from textblob import TextBlob
#Create a Sample DataFrame for Testing
IRA_01 = pd.read_csv('data/IRAhandle_tweets_1.csv')
IRA_01_filtered = IRA_01[IRA_01['retweet']== 0]
sample_df = IRA_01_filtered.sample(n=100, random_state=42)

sample_df

,236016,29642,151659,227463,13678,156374,93741,84339,83344,152250,...,53798,238492,243401,137155,237429,142341,240676,196987,236559,39320
external_author_id,2944766250,893000000000000000,2878166306,895000000000000000,1709314117,2878166306,895000000000000000,3075888895,893000000000000000,2878166306,...,802673000000000000,2944766250,895000000000000000,895000000000000000,2944766250,871000000000000000,2944766250,87588938,2944766250,1657833811
author,ATLANTA_ONLINE,ADNNELSTR,ANDYHASHTAGGER,ARTEMSSFE,AANTIRACIST,ANDYHASHTAGGER,AMBERRPAR,ALTMANBELINDA,ALLBERRTOSS,ANDYHASHTAGGER,...,ALBUQUERQUEON,ATLANTA_ONLINE,AURRLISTR,ANAALESSIS,ATLANTA_ONLINE,ANDIOSMARRTRUMP,ATLANTA_ONLINE,ANZGRI,ATLANTA_ONLINE,ADRIENNE_GG
content,Roswell switches to monthly billing cycle #new...,You can point this out to protesters but they ...,Why would you say that??? https://t.co/hfUzVPLe5e,Benghazi Liar Susan Rice's Treachery Continues...,RT @LatuffCartoons: Rabbi Rabbi Susan Talve: #...,lying on the beach #MyDoomsdayPlans http://t.c...,PremierKevin22 Rus69X VivaLaAmes8 He also appo...,#TrumpBecause USA really needs such leader �,RT johncardillo: Never forget that Democrats a...,#MyNewSingleIsCalled I bought it and now I am ...,...,Albuquerque police officer honored for saving ...,Rome police officers buy groceries for teen ca...,RT 3lectric5heep: Ann Coulter DESTROYS Mitt Ro...,#Ame WATCH: #AltLeft Terror Group Antifa Beats...,Stricter rules for city creation pass Georgia ...,"Idiot Kathy Griffin Attacks Trump AGAIN, Gets ...",Woman dies after shooting in Waffle House park...,'@TMLN_doc это же инсульт',"House Republicans, Georgia delegation mull bud...",The 60s #ThingsInventedWhileHigh
region,United States,Unknown,United States,Unknown,United States,United States,Unknown,United States,Unknown,United States,...,Unknown,United States,United States,Unknown,United States,Unknown,United States,United States,United States,United States
language,English,English,English,English,Arabic,English,English,English,English,English,...,English,English,English,English,English,English,English,Russian,English,English
publish_date,12/24/2015 1:15,8/13/2017 21:16,5/18/2016 15:21,8/12/2017 23:19,2/10/2016 14:43,8/30/2015 15:17,8/16/2017 22:44,8/13/2015 16:04,8/13/2017 3:02,5/4/2016 8:40,...,12/3/2016 0:12,3/21/2017 12:05,8/17/2017 7:54,8/17/2017 10:03,2/29/2016 22:34,6/30/2017 16:49,6/13/2017 9:40,12/20/2015 16:00,2/13/2016 5:36,9/5/2016 11:39
harvested_date,12/24/2015 1:15,8/13/2017 21:16,5/18/2016 15:22,8/12/2017 23:19,2/10/2016 14:43,9/26/2015 5:23,8/16/2017 22:45,8/13/2015 16:04,8/13/2017 3:03,5/4/2016 8:40,...,12/3/2016 0:12,3/21/2017 12:05,8/17/2017 7:54,8/17/2017 10:03,2/29/2016 22:34,6/30/2017 16:50,6/13/2017 9:40,12/20/2015 16:00,2/13/2016 5:36,9/5/2016 11:39
following,9336,946,18011,10,1027,12762,22,50,19,15928,...,60,10917,775,1004,10957,2848,12017,35,10821,1380
followers,14512,83,21732,4,290,13371,7,60,2,21054,...,3,18053,2,138,15899,1046,18653,1278,15433,1089
updates,7094,1991,9188,860,806,2570,937,405,376,8167,...,159,7747,1080,1110,9430,1264,9039,57025,8835,1592


In [97]:
#Use a simple sentiment analysis libary and infer reliability from critical thinking analysis

def analyze_sentiment(text):
    # Create a TextBlob object
    blob = TextBlob(text)
    # Get sentiment polarity
    polarity = blob.sentiment.polarity
    # Classify sentiment based on polarity
    if polarity > 0:
        return "Positive"
    elif polarity < 0:
        return "Negative"
    else:
        return "Neutral"

#sample_df['sentiment'] = sample_df['content'].apply(analyze_sentiment)

# Did not return satifactory analysis
#Created CSV file from it, manual inputed values for sentiment analysis to train BERT

NameError: name 'df_sample' is not defined

In [98]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Load your existing DataFrame
df = pd.read_csv('emotion_intent.csv')

# Convert sentiment labels to numerical values
df['sentiment'] = df['sentiment'].map({'Neutral': 2, 'Positive': 1, 'Negative': 0})

# Split DataFrame into training and evaluation datasets
train_df, eval_df = train_test_split(df, test_size=0.2, random_state=42)

# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

# Set device to CPU
device = torch.device("cpu")
model.to(device)

# Function to perform sentiment analysis
def predict_sentiment(text):
    # Tokenize input text
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    # Move inputs to CPU
    inputs = {key: tensor.to(device) for key, tensor in inputs.items()}
    # Perform sentiment analysis
    outputs = model(**inputs)
    # Get predicted sentiment
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
    return "Negative" if predicted_class == 0 else "Positive"

emotion_df = []
# Function to analyze DataFrame for sentiment
def analyze_dataframe(df):
    # Apply sentiment analysis function to 'content' column
    df['sentiment'] = df['content'].apply(predict_sentiment)
    return df

# Perform sentiment analysis on evaluation DataFrame
eval_df_analyzed = analyze_dataframe(eval_df)

# Print the first few rows of the analyzed DataFrame
print(eval_df_analyzed.head())

print(analyze_dataframe(sample_df))
df.T

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


    external_author_id         author  \
83          1679279490  AMELIEBALDWIN   
53  895000000000000000      ARTEMSSFE   
70  895000000000000000      AMBERRPAR   
45          2500690416    ANNIEPOSHES   
44  895000000000000000      ARTEMSSFE   

                                              content         region language  \
83  #TrumpBecause he’s not afraid of telling the t...  United States  English   
53  Investigate Hillary's Uranium One Collusion wi...        Unknown  English   
70  RT FaithGoldy: #Charlottesville DOUBLE STANDAR...        Unknown  English   
45    #MyOlympicSportWouldBe not believing to Hillary  United States  English   
44  Rescind ObamaCare Rule Exempting Hypocrites In...        Unknown  English   

       publish_date   harvested_date  following  followers  updates  ...  \
83  8/13/2015 17:38  8/13/2015 17:38        492        369     1626  ...   
53  8/12/2017 20:13  8/12/2017 20:13         10          4      836  ...   
70   8/13/2017 1:25   8/13/2017 1:26  

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
external_author_id,2944766250,893000000000000000,2878166306,895000000000000000,1709314117,2878166306,895000000000000000,3075888895,893000000000000000,2878166306,...,802673000000000000,2944766250,895000000000000000,895000000000000000,2944766250,871000000000000000,2944766250,87588938,2944766250,1657833811
author,ATLANTA_ONLINE,ADNNELSTR,ANDYHASHTAGGER,ARTEMSSFE,AANTIRACIST,ANDYHASHTAGGER,AMBERRPAR,ALTMANBELINDA,ALLBERRTOSS,ANDYHASHTAGGER,...,ALBUQUERQUEON,ATLANTA_ONLINE,AURRLISTR,ANAALESSIS,ATLANTA_ONLINE,ANDIOSMARRTRUMP,ATLANTA_ONLINE,ANZGRI,ATLANTA_ONLINE,ADRIENNE_GG
content,Roswell switches to monthly billing cycle #new...,You can point this out to protesters but they ...,Why would you say that??? https://t.co/hfUzVPLe5e,Benghazi Liar Susan Rice's Treachery Continues...,RT @LatuffCartoons: Rabbi Rabbi Susan Talve: #...,lying on the beach #MyDoomsdayPlans http://t.c...,PremierKevin22 Rus69X VivaLaAmes8 He also appo...,#TrumpBecause USA really needs such leader �,RT johncardillo: Never forget that Democrats a...,#MyNewSingleIsCalled I bought it and now I am ...,...,Albuquerque police officer honored for saving ...,Rome police officers buy groceries for teen ca...,RT 3lectric5heep: Ann Coulter DESTROYS Mitt Ro...,#Ame WATCH: #AltLeft Terror Group Antifa Beats...,Stricter rules for city creation pass Georgia ...,"Idiot Kathy Griffin Attacks Trump AGAIN, Gets ...",Woman dies after shooting in Waffle House park...,'@TMLN_doc это же инсульт',"House Republicans, Georgia delegation mull bud...",The 60s #ThingsInventedWhileHigh
region,United States,Unknown,United States,Unknown,United States,United States,Unknown,United States,Unknown,United States,...,Unknown,United States,United States,Unknown,United States,Unknown,United States,United States,United States,United States
language,English,English,English,English,Arabic,English,English,English,English,English,...,English,English,English,English,English,English,English,Russian,English,English
publish_date,12/24/2015 1:15,8/13/2017 21:16,5/18/2016 15:21,8/12/2017 23:19,2/10/2016 14:43,8/30/2015 15:17,8/16/2017 22:44,8/13/2015 16:04,8/13/2017 3:02,5/4/2016 8:40,...,12/3/2016 0:12,3/21/2017 12:05,8/17/2017 7:54,8/17/2017 10:03,2/29/2016 22:34,6/30/2017 16:49,6/13/2017 9:40,12/20/2015 16:00,2/13/2016 5:36,9/5/2016 11:39
harvested_date,12/24/2015 1:15,8/13/2017 21:16,5/18/2016 15:22,8/12/2017 23:19,2/10/2016 14:43,9/26/2015 5:23,8/16/2017 22:45,8/13/2015 16:04,8/13/2017 3:03,5/4/2016 8:40,...,12/3/2016 0:12,3/21/2017 12:05,8/17/2017 7:54,8/17/2017 10:03,2/29/2016 22:34,6/30/2017 16:50,6/13/2017 9:40,12/20/2015 16:00,2/13/2016 5:36,9/5/2016 11:39
following,9336,946,18011,10,1027,12762,22,50,19,15928,...,60,10917,775,1004,10957,2848,12017,35,10821,1380
followers,14512,83,21732,4,290,13371,7,60,2,21054,...,3,18053,2,138,15899,1046,18653,1278,15433,1089
updates,7094,1991,9188,860,806,2570,937,405,376,8167,...,159,7747,1080,1110,9430,1264,9039,57025,8835,1592
